In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [ ]:
crime_data = pd.read_csv("./datasets/crawled_data.csv")
crime_data.head()

In [ ]:
crime_data.info() #Get all the type of each column

In [ ]:
crime_data.describe()

In [ ]:
# Create a Year column to get more generality of the dataset
crime_data["Date"] = pd.to_datetime(crime_data["Date"], format='%m/%d/%Y %I:%M:%S %p')
crime_data["Year"] = crime_data["Date"].dt.year
crime_data["Month"] = crime_data["Date"].dt.month

# crime_data = crime_data.drop(columns=["Location"])

typeToDrop = {
    'NON-CRIMINAL',
    'OTHER OFFENSE',
    'OTHER NARCOTIC VIOLATION',
    'OBSCENITY',
    'PUBLIC INDECENCY',
    'CONCEALED CARRY LICENSE VIOLATION',
    'LIQUOR LAW VIOLATION',
    'GAMBLING',
    'RITUALISM',
}
crime_data = crime_data[~crime_data['Primary Type'].isin(typeToDrop)]
crime_data.columns



In [ ]:
# --- OVERVIEW OF THE CRIME ---

plt.figure(figsize=(30, 10))

crime_types = crime_data.groupby('Primary Type', as_index=False).size()
assault_types = crime_data[crime_data['Primary Type'] == 'ASSAULT'].groupby('Description', as_index=False).size()
auto_theft_types = crime_data[crime_data['Primary Type'] == 'MOTOR VEHICLE THEFT'].groupby('Description', as_index=False).size()

# plt.subplot(221)
sns.barplot(x='Primary Type', y='size', data=crime_types.sort_values(by='size', ascending=False))
plt.title('Crime Types Overview', fontsize='xx-large')
plt.xlabel('Primary Type', fontsize='x-large')
plt.ylabel('Crime Count', fontsize='x-large')
plt.xticks(rotation=90)

# plt.tight_layout()
plt.show()


In [ ]:
# --- OVERVIEW OF MOTOR VEHICLE THEFT ---

# plt.subplot(223)
sns.barplot(x='Description', y='size', data=auto_theft_types.sort_values(by='size', ascending=False))
plt.title('Description Distribution in MOTOR VEHICLE THEFT', fontsize='xx-large')
plt.xlabel('Description', fontsize='x-large')
plt.ylabel('Count', fontsize='x-large')
plt.xticks(rotation=90)
# plt.tight_layout()

In [ ]:
#  --- OVERVIEW OF ASSAULT ---

sns.barplot(x='Description', y='size', data=assault_types.sort_values(by='size', ascending=False))
plt.title('Description Distribution in ASSAULT', fontsize='xx-large')
plt.xlabel('Description', fontsize='x-large')
plt.ylabel('Count', fontsize='x-large')
plt.xticks(rotation=90)

In [ ]:
# --- PLOT BY YEAR ---

yearly_crimes = crime_data.groupby('Year').size().reset_index(name='Total Crimes')
plt.figure(figsize=(15, 6))
sns.barplot(x='Year', y='Total Crimes', data=yearly_crimes, palette='Blues_d')

plt.title('Total Crimes per Year in Chicago', fontsize=20)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Crimes', fontsize=14)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# --- PLOTTING MONTHLY CRIME IN FROM 2020 TO 2024 ---

# Map all the month in word format
month_dict = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}

# Data filter from crime_data
crime_distribution = crime_data.groupby(by=["Year", "Month"], dropna=False, as_index=False).size()
crime_distribution = crime_distribution[crime_distribution["Year"] < 2025]
crime_distribution.rename(columns={'size': "Crime Count"}, inplace=True)
crime_distribution['Year'] = crime_distribution["Year"].astype(str)
crime_distribution["Month"] = crime_distribution["Month"].map(month_dict)
crime_distribution["MonthAndYear"] = crime_distribution["Month"] + ", " + crime_distribution["Year"]


x = np.arange(len(crime_distribution))  # Numeric index for months (0 to length-1)
coefficients = np.polyfit(x, crime_distribution["Crime Count"], deg=1)  # Linear fit (slope and intercept)
trend_line = np.polyval(coefficients, x)  # Evaluate the linear fit over x


# Plot the time series
plt.figure(figsize=(20,10))
plt.grid('on')
plt.plot(crime_distribution["MonthAndYear"], crime_distribution["Crime Count"], color="red")
plt.plot(crime_distribution["MonthAndYear"], trend_line, color="blue", label="Linear Trend", linewidth=2)
plt.xlabel("Month Stream")
plt.ylabel("Total crime count per month")
plt.xticks(np.arange(len(crime_distribution["Month"])), rotation=90)
plt.title("Monthly Crime Trends from 2020 to 2024")
plt.show()


In [ ]:
# --- PLOTTING THE DATA IN A HEATMAP ---
table = crime_distribution.pivot_table(index="Year", columns="Month", values="Crime Count")
plt.figure(figsize=(14, 6))
sns.heatmap(table, annot=True, fmt=".0f", cmap="YlOrRd", linewidths=0.5)
plt.title("Monthly Crime Trends from 2020 to 2024")
plt.xlabel("Month")
plt.ylabel("Year")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# --- PLOTTING CRIME RATE BASED ON LOCATION DESCRIPTION ---

# Function to generalize all the specific location description
def generalize_location(loc_desc):
  # Handle none values
  if pd.isna(loc_desc):
    return "Other"
  loc_desc = str(loc_desc).upper()
  # Residential
  if any(keyword in loc_desc for keyword in ["RESIDENCE", "APARTMENT", "HOUSE", "CHA APARTMENT", "RESIDENTIAL", "PORCH", "YARD", "GARAGE", "BASEMENT", "VESTIBULE", "HALLWAY"]):
      return "Residential"
  # Public Transportation
  elif any(keyword in loc_desc for keyword in ["CTA", "TRAIN", "BUS", "SUBWAY", "PLATFORM", "STATION", "RAILROAD"]):
      return "Public Transportation"
  # Commercial
  elif any(keyword in loc_desc for keyword in ["STORE", "RESTAURANT", "BAR ", "TAVERN", "HOTEL", "MOTEL", "BANK", "CURRENCY EXCHANGE", "GAS STATION", "OFFICE", "WAREHOUSE", "FACTORY", "LIQUOR", "BARBER", "BEAUTY SALON", "PAWN SHOP", "ATM", "CASINO"]):
      return "Commercial"
  # Street/Outdoor
  elif any(keyword in loc_desc for keyword in ["STREET", "SIDEWALK", "ALLEY", "PARKING LOT", "VACANT LOT", "PARK PROPERTY", "LAKEFRONT", "RIVERBANK", "FOREST PRESERVE", "BRIDGE", "HIGHWAY", "GANGWAY", "DRIVEWAY"]):
      return "Street/Outdoor"
  # Institutional
  elif any(keyword in loc_desc for keyword in ["SCHOOL", "HOSPITAL", "CHURCH", "SYNAGOGUE", "GOVERNMENT", "LIBRARY", "POLICE", "FIRE STATION", "NURSING HOME", "DAY CARE", "COLLEGE", "UNIVERSITY"]):
      return "Institutional"
  # Airport
  elif any(keyword in loc_desc for keyword in ["AIRPORT", "AIRCRAFT"]):
      return "Airport"
  # Vehicle
  elif any(keyword in loc_desc for keyword in ["VEHICLE", "AUTO", "TAXICAB", "UBER", "LYFT", "TRUCK", "BOAT"]):
      return "Vehicle"
  # Other
  else:
      return "Other"


crime_data["Generalized Location"] = crime_data["Location Description"].apply(generalize_location)


# Data filter from crime_data
crime_rate_based_on_loc_desc = crime_data.groupby(by=['Generalized Location'], dropna=False, as_index=False).size()
crime_rate_based_on_loc_desc.rename(columns={"size": "count", "Generalized Location": "loc_desc"}, inplace=True)

# Sort the values before plotting
crime_rate_based_on_loc_desc.sort_values("count", ascending=False)

# Plot the crime based on location description
plt.figure(figsize=(20,10))
sns.barplot(x="loc_desc", y="count", data=crime_rate_based_on_loc_desc, palette='Blues_d')
plt.xlabel("Location description")
plt.ylabel("Total crime for each loaction description")
plt.xticks(rotation=90)
plt.title("Crime Trends based on different location")
plt.show()


# set(crime_rate_based_on_loc_desc["loc_desc"].tolist())

In [ ]:
# Install pydeck for population plot
%pip install pydeck

In [ ]:
# --- PLOTTING THE CRIME TREND BASED ON LOCATION IN CHICAGO ---

import pydeck as pdk

crime_data['Lat_round'] = crime_data['Latitude'].round(3)
crime_data['Lon_round'] = crime_data['Longitude'].round(3)
crime_location_counts = crime_data.groupby(['Lat_round', 'Lon_round']).size().reset_index(name='Count')

# Function to assign colors for better visualization
def assign_color(count):
    if count <= 100:
        return [0, 0, 255, 255]  # Blue for low counts (0-100)
    elif count <= 500:
        return [0, 255, 0, 255]  # Green for medium counts (101-500)
    else:
        return [255, 0, 0, 255]  # Red for high counts (501+)

# Apply the color breakpoints to the DataFrame
crime_location_counts['color'] = crime_location_counts['Count'].apply(assign_color)

# Set up the layer for the map
layer = pdk.Layer(
    type="ColumnLayer",
    data=crime_location_counts,
    get_position=["Lon_round", "Lat_round"],
    get_elevation="Count",
    auto_hightlight =True,
    elevation_scale=0.1,
    pickable=True,
    get_fill_color=["Count", 255],
    coverage=10
)

# Set the view parameter
view_state = pdk.ViewState(
    latitude=41.8781,    
    longitude=-87.6298,
    zoom=10,
    pitch=0
)

# Render the map to a html file
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"html": "<b>Crime Count:</b> {Count}<br/><b>Location:</b> [{Lat_round}, {Lon_round}]"}
)
r.to_html("chicago_crime_map.html")